# Часть 1. Парсинг.

Возьмём первые n акций со страницы https://smart-lab.ru/q/shares/ и скачаем P/E компаний за последний период.

In [9]:
#!pip install fake_useragent
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm import tqdm

In [13]:
n = 100 # Сколько акций будем скачивать
delay = 1 # delay for requests (in seconds)
tickers = []
url = 'https://smart-lab.ru/q/shares/'
pattern = r'^/forum/([A-Z]+)$'

def get_soup(url):
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    html_text = response.text
    soup = BeautifulSoup(html_text, 'html.parser')
    time.sleep(delay)
    return soup

soup = get_soup(url)

for a in soup.find_all('a'):
    match = re.match(pattern, a.get('href'))
    if match:
        tickers.append(match.group(1))
        if len(tickers) == n:
            break

print(tickers[:10], '...')

['SBER', 'LKOH', 'TCSG', 'GAZP', 'PLZL', 'YNDX', 'AFKS', 'MTLR', 'OZON', 'VTBR'] ...


In [14]:
pe_list = []
descriptions = []
url_base = 'https://smart-lab.ru'

for t in tqdm(tickers):
    soup = get_soup(url_base + r'/q/' + t + r'/f/y/')
    pe_row = soup.find('tr', attrs = {'field': 'p_e'})
    td_elements = pe_row.find_all("td", class_=None)
    pe_list.append(td_elements[-1].text.strip())

    descriptions.append(soup.find('div', attrs={'class': 'company_description'}).text)

df_pe_by_ticket = pd.DataFrame(columns=['Ticket', r'P/E', 'Description'])

for i, (t, pe, desc) in enumerate(zip(tickers, pe_list, descriptions)):
    desc = desc.strip().replace('\n\n\n', '')

    if i < 3:
        print(t, pe)
        print(desc)
        print()

    desc_in_line = desc.replace('\t', '').replace('\n', ' ').replace('"', '')
    desc_in_line = desc_in_line.replace('|', '').replace(',', '')
    data_row = {'Ticket': t, 
                r'P/E': pe, 
                'Description': desc_in_line}
    df_pe_by_ticket = pd.concat([df_pe_by_ticket, pd.DataFrame([data_row])], 
                                ignore_index=True).dropna(axis=1)
    
df_pe_by_ticket.to_csv('pe_by_ticket.csv', index=False)
df_pe_by_ticket

100%|██████████| 100/100 [04:38<00:00,  2.78s/it]

SBER 3.52
Сбербанк - факторы роста и падения акций
			
Сбербанк перешел на выплату дивидендов 50% от прибыли начиная с 2020 года
Сбербанк вышел в прибыль в октябре 2022 года и может выплатить дивиденды уже в 2023 году
Рекордная прибыль в 2023 году и ожидаемый рекордный дивиденд.
Могут платить больше 50% от чистой прибыли. Высокий ROE и высокая достаточность капитала.

Замедление кредитования в стране снижает рост кредитного портфеля и соответственно процентных доходов Сбера.
Рост процентных ставок может снизить чистую процентную маржу и соответственно прибыль Сбера в следующем году.
Ипотека - основа розничного кредитного портфеля. Средние сроки ипотечного кредита в среднем выросли за последние год на 10 лет - вырос риск, что со временем могут начаться проблемы с выплатой.

LKOH 3.97
Лукойл - факторы роста и падения акций
			
Стабильно растущие дивиденды + див. политика, к-я предусматривает выплату 100% скорр. свободного денежного потока на дивиденды
Дивидендная доходность Лукойла может

,Ticket,P/E,Description
0,SBER,3.52,Сбербанк - факторы роста и падения акций Сбер...
1,LKOH,3.97,Лукойл - факторы роста и падения акций Стабил...
2,TCSG,5.34,Т-Банк Тинькофф ТКС - факторы роста и падени...
3,GAZP,-5.01,Газпром - факторы роста и падения акций Могут...
4,PLZL,12.9,Полюс - факторы роста и падения акций Себесто...
...,...,...,...
95,HNFG,7.17,Henderson (Хэндерсон фэшн групп) - факторы рос...
96,AKRN,17.4,Акрон - факторы роста и падения акций Низкий...
97,CIAN,19.6,ЦИАН - факторы роста и падения акций
98,GEMC,5.73,ЕМС ЮМГ - факторы роста и падения акций Цены...
